In [1]:
"""
LIBRARIES IMPORTED
"""
import pandas as pd
import numpy as np
from pandas import DataFrame
from graphviz import Digraph
from locationiq.geocoder import LocationIQ
from math import radians,sin,cos,acos
import json

In [2]:
"""
BACKTRACE FUNCTION TO GET THE FINAL ABSOLUTE PATH
"""
def backtrace(explored):
    backtrace=[]
    node=explored[-1]
    while node!=None:
        backtrace.append(node)
        node=node.parent
    
    return backtrace[::-1]

In [3]:

"""
VIZUALIZATION  
"""

#VIZIUALIZATION FUNCTION FOR BFS/DFS/BDBFS
def createVizual1(explored,goal):
    
    path=backtrace(explored)
    
    graph=Digraph('Graph-Problem1')
    done=[]
    
    for node in explored:
        if node in path:
            graph.node(node.state,color='red',shape='doublecircle',style='filled', fillcolor='lightgrey')
        else:
            graph.node(node.state)
            
        
        for child in node.result_nodes:
            if child not in done:
                if search_type.lower()=="a*":
                    graph.edge(node.state,child.state,label='f(n): '+str(round(child.fvalue,2)))
                elif search_type.lower()=="gbfs":
                    graph.edge(node.state,child.state,label='f(n): '+str(round(child.hvalue,2)))
                else:
                    graph.edge(node.state,child.state,label='Cost: '+str(round(child.gvalue,2)))
            
            done.append(child)
    
    graph.render('Graph-Problem1.gv', view=True)

In [4]:
"""
MISC FUNCTIONS
"""

#RETURNS THE PATH COST BETWEEN TWO NODES FROM THE READ FILE
def find_cost(file,s,d):
    cost=999999
    for i in file.index:
        if(file[0][i]==s and file[1][i]==d):
            cost=file[2][i]
            break
        elif(file[1][i]==s and file[0][i]==d):
            cost=file[2][i]
            break    
    return cost


#RETURNS THE HEURISTIC VALUES OF ALL THE CITY NODES
def hvalue(goal):
#     geocoder=LocationIQ("92766540a2f263")
    geocoder=LocationIQ("323ef67c648e78")
#     geocoder=LocationIQ("715e7b2c0dacae")
    to=geocoder.geocode(goal)[0]
    elat=radians(float(to['lat']))
    elon=radians(float(to['lon']))
    
    dist=dict()
    print("(",end=" ")
    for i in df.index:
        frm=geocoder.geocode(i)[0]
        slat=radians(float(frm['lat']))
        slon=radians(float(frm['lon']))
        
        print(i,end=", ")
        dist[i]=6371.01*acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon-elon))
    
    print(")",end=" ")    
    return dist



# ADDS THE LIST OF CAN-GO CITIES FOR THE CURRENT NODE 
def addCities(current_state):
    children=[]
    for i in df.index:
        if i!=current_state:
            if df[i][current_state]<999999 and df[i][current_state]>0:
                children.append(i)
    
    return children



#RETURNS A CHILD NODE WITH THE MINIMUM EVALUATION FUNCTION
def renderMinFnode(node,explored):
    
    minnode=None
    minfvalue=999999
    for child in node.result_nodes:
        if (child not in explored) and (child.fvalue<minfvalue):
            minfvalue=child.fvalue
            minnode=child
    
    return minnode


#RETURNS A CHILD NODE WITH THE MINIMUM HEURISTIC FUNCTION
def renderMinHnode(node,explored):
    
    minnode=None
    minhvalue=999999
    for child in node.result_nodes:
        if (child not in explored) and (child.hvalue<minhvalue):
            minhvalue=child.hvalue
            minnode=child
    
    return minnode


"""
MISC FUNCTIONS FOR 8 PUZZLE PROBLEM
"""

def getIndex(current_state):
    for i in range(N):
        for j in range(N):
            if current_state[i][j]==0:
                return (i,j)

def getHvalue(current_state,goal_state):
    count=0
    for i in range(N):
        for j in range(N):
            if current_state[i][j]!=goal_state[i][j]:
                count=count+1
    
    return count



def moveRight(current_state,i,j):
    newState=current_state.copy()
    newState[i][j]=newState[i][j+1]
    newState[i][j+1]=0
    
    return newState

def moveLeft(current_state,i,j):
    newState=current_state.copy()
    newState[i][j]=current_state[i][j-1]
    newState[i][j-1]=0
    
    return newState


def moveDown(current_state,i,j):
    newState=current_state.copy()
    newState[i][j]=current_state[i+1][j]
    newState[i+1][j]=0
    
    return newState

def moveUp(current_state,i,j):
    newState=current_state.copy()
    newState[i][j]=current_state[i-1][j]
    newState[i-1][j]=0
    
    return newState


def addPuzzle(current_state):
    index=getIndex(current_state)
    i=index[0]
    j=index[1]
    
    children=[]
    
    if i==0:
        if j==0:
            #move right
            children.append(moveRight(current_state,i,j))
            
        elif j==1:
            #move right and left
            children.append(moveRight(current_state,i,j))
            children.append(moveLeft(current_state,i,j))
            
        elif j==2:
            #move left
            children.append(moveLeft(current_state,i,j))
        
        #default move down
        children.append(moveDown(current_state,i,j))
    
    elif i==1:
        if j==0:
            #move up,right,down
            children.append(moveRight(current_state,i,j))            
            
        elif j==1:
            #move in all
            children.append(moveRight(current_state,i,j))
            children.append(moveLeft(current_state,i,j))
            
        elif j==2:
            #move up,down,left
            children.append(moveLeft(current_state,i,j))
        
        #default move up, down
        children.append(moveDown(current_state,i,j))
        children.append(moveUp(current_state,i,j))
    
    elif i==2:
        if j==0:
            #move up,right
            children.append(moveRight(current_state,i,j))
            
            
        elif j==1:
            #move up,right,left
            children.append(moveRight(current_state,i,j))
            children.append(moveLeft(current_state,i,j))
            
        elif j==2:
            #move left,up
            children.append(moveLeft(current_state,i,j))
            
        #default move up
        children.append(moveUp(current_state,i,j))
        
    return children
        

In [5]:
"""
THE SEARCH ALGORITHMS
"""

def foundNode(node,found_states):
    for k in found_states:
        if np.array_equal(k,node):
            return True
    
    return False

def TransitionModel(current_node,found_states,goal,choice):
    for node in current_node.result_states:
        if choice==1:

            if node not in found_states:
                #create new node
                newnode=Node(node,current_node,df[node][current_node.state],goal,choice)
                current_node.result_nodes.append(newnode)
                found_states.append(node)
        else:
            
            if not foundNode(node,found_states):
                newnode=Node(node,current_node,0,goal,choice)
                current_node.result_nodes.append(newnode)
                found_states.append(node)
                
                
        
    
#BREADTH FIRST SEARCH
def bfs(start_node,GoalTest):
    queue=[]
    queue.append(start_node)
        
    explored=[]
    found_states=[start_node.state]
    
    while queue:
        present=queue.pop(0)
        if not GoalTest(goal,present,choice):
#             print(present.state)
            TransitionModel(present,found_states,goal,choice)
            queue.extend(present.result_nodes)
            explored.append(present)
        else:
            explored.append(present)
#             print('Passed Test!YOHOO')
            break
    
    
    return explored



#DEPTH FIRST SEARCH

def dfs(start_node,GoalTest):
    explored=[]
    stack=[]
        
    stack.append(start_node)
    found_states=[start_node.state]
        
    while stack:
        present=stack.pop(-1)
        if not GoalTest(goal,present,choice):
#             print(present.state)
            TransitionModel(present,found_states,goal,choice)
            stack.extend(present.result_nodes)
            explored.append(present)
        else:
            explored.append(present)
            break
    
    return explored


#BIDIRECTIONAL BFS

def bdbfs(start_node,goal_node,GoalTest):
    
    fqueue=[]
    bqueue=[]

    fexplored=[]
    bexplored=[]
        
    fqueue.append(start_node)
    bqueue.append(goal_node)
    
    ffound=[start_node.state]
    bfound=[goal_node.state]
    
    while fqueue:
        fcurrent=fqueue.pop(0)
        fexplored.append(fcurrent)
            
        if GoalTest(goal,fcurrent,choice):
            break
        else:
            TransitionModel(fcurrent,ffound,goal_node.state,choice)
            fqueue.extend(fcurrent.result_nodes)
        
    while bqueue:
        bcurrent=bqueue.pop(0)
        bexplored.append(bcurrent)
            
        if GoalTest(current_node.state,bcurrent,choice):
            break
        else:
            TransitionModel(bcurrent,bfound,start_node.state,choice)
            bqueue.extend(bcurrent.result_nodes)
        

        
    for i in range(len(fexplored)):
        flag=0
        for j in range(len(bexplored)):
            if (fexplored[i].state!=current_node.state) and (bexplored[j].state!=goal_node.state):
                if fexplored[i].state==bexplored[j].state:
                    flag=1
                    break
        if flag==1:
            break
        

    explored=[]
    k=0
    while(k<i):
        explored.append(fexplored[k])
        k=k+1
        
        
    k=j
    
    while(k>=0):
        for n in fexplored:
            if n.state==bexplored[k].state:
                break            
        explored.append(n)
        k=k-1


    return explored





#A* ALGORITHM

def astar(start_node,GoalTest):
#     problem.append(start_node)
    explored=[]
    found_states=[start_node.state]
    current_node=start_node
    
    while not GoalTest(goal,current_node,choice):
#         print(current_node.state)
        explored.append(current_node)
        
        #first create child nodes for current node
        TransitionModel(current_node,found_states,goal,choice)
                
#                 problem.append(newnode)
        
        #we change the current node to the min of child nodes
        
        if renderMinFnode(current_node,explored):
            current_node=renderMinFnode(current_node,explored)
        
        #else if current node has no more possible states to visit, we go back to the parent and so on
        
        else:
            while not renderMinFnode(current_node,explored):
                current_node=current_node.parent        
    
    
    explored.append(current_node)   
    return explored



#GBFS ALGORITHM

def gbfs(start_node,GoalTest):
#     problem.append(start_node)
    explored=[]
    found_states=[start_node.state]
    current_node=start_node
    
    while not GoalTest(goal,current_node,choice):
#         print(current_node.state)
        explored.append(current_node)
        
        TransitionModel(current_node,found_states,goal,choice)
                
        
        if renderMinHnode(current_node,explored):
            current_node=renderMinHnode(current_node,explored)
        
        else:
            while not renderMinHnode(current_node,explored):
                current_node=current_node.parent        
    
    
    explored.append(current_node)   
    return explored

In [6]:

"""
FUNCTIONS FOR CREATING A SIMPLE-PROBLEM SOLVING AGENT
"""

#CLASS FOR CREATING A NODE
class Node:  
    def __init__(self,current_state,parent_node,gvalue,goal,question):
        
        self.state=current_state                          #STATE REPRESENTED BY THE NODE
                
        self.parent=parent_node                           #PARENT NODE
        
        self.result_nodes=[]                              #LIST OF CHILD NODES EXISTING (NODE FORMAT)
        
        if question==1:
            
            self.result_states=addCities(current_state)       #LIST OF POSSIBLE CHILD STATES IN STRING FORMAT
            
            if parent_node==None:
                self.gvalue=0                                 #FOR STARTING NODE

            else:
                self.gvalue=parent_node.gvalue+gvalue         #FOR ALL OTHER NODES
            self.hvalue=hvalues[self.state]              
            self.fvalue=self.gvalue+self.hvalue           #f(n)=g(n)+h(n) for A* Algorithm
            
        
        elif question==2:
            
            self.result_states=addPuzzle(current_state)       #LIST OF POSSIBLE CHILD STATES IN STRING FORMAT
            
            if parent_node==None:
                self.gvalue=0
            
            else:
                self.gvalue=parent_node.gvalue+1
            
            self.hvalue=getHvalue(current_state,goal)     #f(n)=h(n) for Greedy Best First Search
            self.fvalue=self.gvalue+self.hvalue           #f(n)=g(n)+h(n) for A* Algorithm




#GOAL-TEST TO CHECK IF THE CURRENT NODE HAS THE GOAL STATE

def GoalTest(goal,node,question):
    if question==1:
        
        if node.state==goal:
            return True
        else:
            return False
        
    elif question==2:
        
        if np.array_equal(goal,node.state):
            return True
        
        else:
            return False


        
#SEARCH FUNCTION FORMULAATING THE PATH

def Search(start_node,goal,TransitionModel,GoalTest,search_type,question):
    
    
    if search_type.lower()=="bfs":
        explored=bfs(start_node,GoalTest)
        
                
    elif search_type.lower()=="dfs":
        explored=dfs(start_node,GoalTest)
        
    
    elif search_type.lower()=="bdbfs":
        goal_node=Node(goal,None,0,start_node.state,choice)
        
        
        explored=bdbfs(start_node,goal_node,GoalTest)
        
        
    elif search_type.lower()=="a*":
        problem=[]
        explored=astar(start_node,GoalTest)
    
    elif search_type.lower()=="gbfs":
        problem=[]
        explored=gbfs(start_node,GoalTest)

    
    
    if question==1:
        createVizual1(explored,goal)
    
        
    final_sequence=backtrace(explored)
    
    return final_sequence
        
        
    



In [7]:
"""
MAIN FUNCTION
"""

choice=int(input("Enter the problem number:\n1. Taxi-Driver Agent\n2. 8-Puzzle problem\n"))

if choice ==1:
    
    """
    READ THE .XLSX FILE USING PANDAS
    """
    
    file=pd.read_excel('Indian_capitals.xlsx', header=None)

    
    """MAKE A LIST OF ALL CITIES
    WITHOUT DUPLICATES FOR INDEXING"""

    indices=[]
    for i in range(53):
        if file[0][i] not in indices:
            indices.append(file[0][i])
        if file[1][i] not in indices:
            indices.append(file[1][i])


    """CREATE A DATAFRAME -- A MULTI DIMENSIONAL ARRAY 
    WITH THE CITIES AS THE INDEX/COLUMN NAMES
    AND THEIR PATH COSTS AS THE VALUE"""

    df=DataFrame(np.arange(len(indices)*len(indices)).reshape(len(indices),len(indices)), index=indices, columns=indices)
    for i in df.index:
        for j in df.columns:
            if i==j:
                df.loc[j,i]=0
            else:
                df.loc[j,i]=find_cost(file,j,i)
                df.loc[i,j]=df[j][i]
#         print(df)



    start=input('Enter starting place: ').capitalize()

    goal=input('Enter destination place: ').capitalize()

    
    print("\nPLEASE WAIT... RETRIEVING DATA FROM LOCATION IQ\nNOTE: IN CASE OF ANY LOCATION IQ ERROR, RESTART(>>) THE KERNEL IN A MINUTE.\n")
    hvalues=hvalue(goal)
    # print(hvalues)

    current_node=Node(start,None,0,goal,choice)

    search_type=input('Enter Search type(BFS/DFS/BD-BFS/A*)')

    path=Search(current_node,goal,TransitionModel,GoalTest,search_type,choice)
    

elif choice==2:
    
    N=3
    start=np.arange(N*N).reshape((N,N))
    print('Enter the start state of the puzzle: ')
    
    for i in range(N):
        for j in range(N):
            start[i][j]=int(input())
    
    goal=np.arange(N*N).reshape((N,N))
    
    start_node=Node(start,None,0,goal,choice)
    
    print('Enter the goal state of the puzzle: ')
    
    for i in range(N):
        for j in range(N):
            goal[i][j]=int(input())    
    
    search_type=input('Enter Search type(BFS/DFS/BDBFS/GBFS/A*)')
    
    path=Search(start_node,goal,TransitionModel,GoalTest,search_type,choice)
    

    
if search_type.lower()=="a*":
    for i in path:
        print('\n-------New State-------\t',i.state,"\n-------Path Cost-------\t",i.gvalue,"\n-------Evaluation Function Value-------\t",i.fvalue)


elif search_type.lower()=="gbfs":
    for i in path:
        print('\n-------New State-------\t',i.state,"\n-------Path Cost-------\t",i.gvalue,"\n-------Evaluation Function Value-------\t",i.hvalue)

            
else:
    for i in path:
        print('\n-------New State-------\t',i.state,"\n-------Path Cost-------\t",i.gvalue)

    
print("\nTotal Path Cost: ",path[-1].gvalue)

Enter the problem number:
1. Taxi-Driver Agent
2. 8-Puzzle problem
2
Enter the start state of the puzzle: 
0
1
2
3
4
5
6
7
8
Enter the goal state of the puzzle: 
1
2
5
3
4
0
6
7
8
Enter Search type(BFS/DFS/BDBFS/GBFS/A*)a*

-------New State-------	 [[0 1 2]
 [3 4 5]
 [6 7 8]] 
-------Path Cost-------	 0 
-------Evaluation Function Value-------	 0

-------New State-------	 [[1 0 2]
 [3 4 5]
 [6 7 8]] 
-------Path Cost-------	 1 
-------Evaluation Function Value-------	 4

-------New State-------	 [[1 2 0]
 [3 4 5]
 [6 7 8]] 
-------Path Cost-------	 2 
-------Evaluation Function Value-------	 4

-------New State-------	 [[1 2 5]
 [3 4 0]
 [6 7 8]] 
-------Path Cost-------	 3 
-------Evaluation Function Value-------	 3

Total Path Cost:  3
